In [ ]:
import pickle
import json
from pathlib import Path
from slideminer import data

In [ ]:
DATA = Path('/data/DeepLearning/SCHWOB_Robin/Seg_outputs_iter_7')
ITER = 3
TREE = DATA/'Tree/most_significant_tree.p'
PATCHES = DATA/f'Selection_1'

In [ ]:
with open(TREE, 'rb') as f:
    p_tree = pickle.load(f)

In [ ]:
max(p_tree.children.keys()), p_tree.children[max(p_tree.children.keys())]

(16128, [11598, 11611])

In [ ]:
with open(TREE.parent/'full_tree.p', 'rb') as f:
    f_tree = pickle.load(f)

In [ ]:
root = max(f_tree.children.keys())

In [ ]:
root, f_tree.children[root], [f_tree.population[c] for c in f_tree.children[root]]

(16128, (16127, 16126), [3, 8062])

In [ ]:
f_tree.population[11598], f_tree.population[11611]

(525, 2129)

In [ ]:
n = 100
node = root
while n > 0:
    children = sorted(f_tree.children[node], key=f_tree.population.get)
    pops = [f_tree.population[c] for c in children]
    print(f'Node: {node}, children: ' + ', '.join([f'{c} (pop: {pop})' for c, pop in zip(children, pops)]))
    n -= 1
    node = children[-1]

Node: 16128, children: 16127 (pop: 3), 16126 (pop: 8062)
Node: 16126, children: 4559 (pop: 1), 16125 (pop: 8061)
Node: 16125, children: 16060 (pop: 3), 16124 (pop: 8058)
Node: 16124, children: 16122 (pop: 3), 16123 (pop: 8055)
Node: 16123, children: 5521 (pop: 1), 16121 (pop: 8054)
Node: 16121, children: 3083 (pop: 1), 16120 (pop: 8053)
Node: 16120, children: 2634 (pop: 1), 16119 (pop: 8052)
Node: 16119, children: 16110 (pop: 5), 16118 (pop: 8047)
Node: 16118, children: 7802 (pop: 1), 16117 (pop: 8046)
Node: 16117, children: 6882 (pop: 1), 16115 (pop: 8045)
Node: 16115, children: 4349 (pop: 1), 16114 (pop: 8044)
Node: 16114, children: 696 (pop: 1), 16113 (pop: 8043)
Node: 16113, children: 4788 (pop: 1), 16112 (pop: 8042)
Node: 16112, children: 15905 (pop: 2), 16111 (pop: 8040)
Node: 16111, children: 4475 (pop: 1), 16109 (pop: 8039)
Node: 16109, children: 3659 (pop: 1), 16108 (pop: 8038)
Node: 16108, children: 16107 (pop: 17), 16106 (pop: 8021)
Node: 16106, children: 3024 (pop: 1), 1610

In [ ]:
truncs = [int(node.name) for node in PATCHES.iterdir()]

In [ ]:
leaves_by_trunc = data.leaves_under_truncated_leaves(p_tree, truncs)

In [ ]:
all_stats = {}
for concept in PATCHES.iterdir():
    all_stats[int(concept.name)] = {'04': 0, '05': 0, '08': 0}
    for patch in concept.iterdir():
        for pacs in ('04', '05', '08'):
            if f'PACS{pacs}' in patch.name:
                all_stats[int(concept.name)][pacs] += 1
                break

In [ ]:
def get_pacs_stats(concept):
    if concept in all_stats:
        return all_stats[concept]
    else:
        count = {'04': 0, '05': 0, '08': 0}
        for child in p_tree.children[concept]:
            child_count = get_pacs_stats(child)
            for pacs in count:
                count[pacs] += child_count[pacs]
        return count

In [ ]:
def compute_dict(nodes):
    parents = []
    for node in nodes:
        node = int(node)
        try:
            parent = int(p_tree.parents[node])
        except:
            parent = node
        test = node != parent
        if test:
            jsondict['links'].append({'source': parent, 'target': node})
        dico = {'id': node,
                'parent': parent}
        if node in all_stats:
            dico['children'] = []
            count = all_stats[node]
            tot = sum(count.values())
            dico['leaf_size_color'] = [count[p] / tot for p in ('04', '05', '08')] + [1.0]
        else:
            dico['children'] = [int(c) for c in p_tree.children[node] if str(c) in jsondict['nodes']]
            n = len(dico['children'])
            dico['leaf_size_color'] = [0, 0, 0, 0]
            for child in dico['children']:
                child_color = jsondict['nodes'][str(child)]['leaf_size_color']
                for k, c in enumerate(child_color):
                    dico['leaf_size_color'][k] += c/n
        jsondict['nodes'][str(node)] = dico
        if test and parent not in parents:
            parents.append(parent)
    if len(parents):
        compute_dict(parents)

In [ ]:
jsondict = {'links': [], 'nodes': {}}
compute_dict(leaves_by_trunc)

In [ ]:
with open(TREE.with_suffix('.json'), "w") as f:
    json.dump(jsondict, f, indent=4, sort_keys=True)